In [2]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [3]:
df_model = pd.read_pickle('df_model_without_stemming.p')

In [4]:
df_model.head()

,count_words,count_mentions,count_hashtags,count_capital_words,count_excl_quest_marks,count_urls,count_emojis,sentiment,origin_text,likes,retweets,clean_text
557219,20,1,1,0,1,1,0,neutral,5 ways Internet of things will create new opportunities for Indian businesses!Read here http://t.co/NiroT9Xa9I #DigitalUniverseIN @emcindia,0,0,ways internet things create new opportunities indian businesses read digitaluniversein
5084791,21,1,4,0,0,2,0,neutral,"As #bigdata, #cloud, and #IoT usage increases, so do #security concerns https://t.co/lXwuN7CP6s via @ITProPortal https://t.co/8KPv2ZbcuY",0,0,bigdata cloud iot usage increases security concerns
4330044,15,0,1,0,0,1,0,neutral,Hewlett Packard Enterprise Co Launches an Internet of Things Platform https://t.co/qf8qKwAhFs #iot,1,0,hewlett packard enterprise co launches internet things platform iot
4846444,17,0,0,0,0,1,0,neutral,"The Internet of Things is about to disrupt the digital economy, report says https://t.co/RFWzNYvNY1",0,1,internet things disrupt digital economy report says
1338071,14,0,2,0,0,1,0,neutral,How the #InternetOfThings is changing the World around Us http://t.co/iPz10oo7hc … #IoT,0,0,internetofthings changing world around us iot


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [6]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [7]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
    textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    

    features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                            , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)

    
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [8]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for MultinomialNB
parameters_mnb = {
    'clf__alpha': (0.25, 0.5, 0.75)
}

In [9]:
mnb = MultinomialNB()

In [10]:
countvect = CountVectorizer()

In [11]:
# MultinomialNB
best_mnb_countvect = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.8min finished


done in 114.364s

Best CV score: 0.770
Best parameters set:
	clf__alpha: 0.75
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.770


Classification Report Test Data
              precision    recall  f1-score   support

    negative       0.80      0.78      0.79      1360
     neutral       0.77      0.71      0.74      1877
    positive       0.75      0.82      0.78      1716

   micro avg       0.77      0.77      0.77      4953
   macro avg       0.77      0.77      0.77      4953
weighted avg       0.77      0.77      0.77      4953

all results
0.721244 (0.004160) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.762300 (0.001667) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 2)}
0.722926 (0.00

In [14]:
tfidfvect = TfidfVectorizer()

In [15]:
best_mnb_tfidf = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.9min finished


done in 120.795s

Best CV score: 0.752
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.5
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.757


Classification Report Test Data
              precision    recall  f1-score   support

    negative       0.81      0.75      0.78      1360
     neutral       0.74      0.72      0.73      1877
    positive       0.74      0.80      0.77      1716

   micro avg       0.76      0.76      0.76      4953
   macro avg       0.76      0.76      0.76      4953
weighted avg       0.76      0.76      0.76      4953

all results
0.699616 (0.002349) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.740762 (0.003084) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 2)}
0.703273 (0.001

In [16]:
#max_df: 0.25 or maximum document frequency of 25%.
#min_df: 2 or the words need to appear in at least 2 tweets
#ngram_range: (1, 2), both single words as bi-grams are used
#clf__alpha: 0.75
textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    
features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
                                              , ('vect', CountVectorizer(max_df=0.25, min_df=2, ngram_range=(1,2)))]))]
                       , n_jobs=-1)

pipeline = Pipeline([
    ('features', features)
    , ('clf', MultinomialNB(alpha=0.75))
])

best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)

In [17]:
df_model_pos = pd.read_pickle('df_model_pos_stem.p')
best_model.predict(df_model_pos).tolist()

['positive', 'positive', 'positive']

In [19]:
df_model_neg = pd.read_pickle('df_model_neg_stem.p')
best_model.predict(df_model_neg).tolist()

['negative', 'negative', 'negative', 'neutral']